In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import pymysql
from PyQt5 import QtCore, QtGui, QtWidgets
from DATA225utils import make_connection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from PyQt5 import uic
from PyQt5.QtCore import Qt, pyqtSignal
from PyQt5.QtGui import QPixmap,QFont, QPainter, QPalette, QBrush,QIcon

from PyQt5.QtWidgets import QApplication, QSizePolicy, QFrame, QComboBox, QMainWindow, QVBoxLayout, QWidget, QLabel, QPushButton, QHeaderView, QRadioButton, QDialog, QLineEdit, QSpacerItem, QTableWidget, QTableWidgetItem, QMessageBox, QHBoxLayout



In [2]:
class ClickableLabel(QLabel):
    clicked = pyqtSignal()

    def __init__(self, parent=None):
        super(ClickableLabel, self).__init__(parent)
        self.setCursor(Qt.PointingHandCursor)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.clicked.emit()

## Login Page

In [3]:
class LoginPage(QDialog):

    
    def __init__(self):
    
        super().__init__()
        uic.loadUi('LoginPage.ui', self)
        
        self.textbox_password.setEchoMode(QLineEdit.Password)
        
        
        
        self.background_pixmap = QPixmap('background_image.png')

        self.paintEvent(self.background_pixmap)
        
       

        self.RegisterPage = RegisterPage()
        self.EmpLoginPage = EmpLoginPage()
        self.employee_welcome_page = employee_welcome_page()
        self.WelcomePage = WelcomePage()
        
        

        
        self.horizontal_layout = QHBoxLayout()
        self.container_widget = QWidget(self)
        self.container_widget.setLayout(self.horizontal_layout)


        self.non_clickable_label = QLabel("Employee? ", self)
        self.non_clickable_label.move(780, 670)  
        self.non_clickable_label.setStyleSheet("color: white;")  
        font = QFont()
        font.setPointSize(13)  
        self.non_clickable_label.setFont(font)


        # Clickable label
        self.clickable_label = ClickableLabel(self)
        self.clickable_label.setText("login here")
        self.clickable_label.move(850, 670)  
        self.clickable_label.setStyleSheet("color: rgb(186, 216, 255);")   
        self.clickable_label.setFont(font)  
        self.clickable_label.clicked.connect(self._show_emplogin)


        
        self.button_login.clicked.connect(self.on_login)
        self.button_create_login.clicked.connect(self._show_register_page)
        
        
    def paintEvent(self,event):
        painter = QPainter(self)
        painter.drawPixmap(self.rect(), self.background_pixmap)
        
        

    def _show_register_page(self):
        self.RegisterPage.show_dialog()
        
    def _show_welcome_page(self,username):
        self.WelcomePage = WelcomePage()
        self.WelcomePage.set_username(username)
        self.WelcomePage._show_oldorderstable(username)
        self.WelcomePage.show_dialog()

    def _show_emplogin(self):

        self.EmpLoginPage.show_dialog()

    def on_login(self):
        username = self.textbox_username.text()
        password = self.textbox_password.text()
        conn = make_connection('invictus_db.ini')
        cursor = conn.cursor()
        cursor.execute("SELECT _password FROM cred WHERE username = %s", (username,))
        result = cursor.fetchone()

        if result and result[0] == password:
            QMessageBox.information(self, 'Login', 'Successful login!')
           
            self._show_welcome_page(username)

        else:
            QMessageBox.warning(self, 'Login', 'Incorrect username or password.')
    
    
    def show_dialog(self):
        self.exec_()


# Registration Page

In [4]:
class RegisterPage(QDialog):
    def __init__(self):
        super().__init__()
        uic.loadUi('RegisterPage.ui', self)
        
        self.button_register.clicked.connect(self.on_register)
        
        self.background_pixmap = QPixmap('background_image.png')

        self.paintEvent(self.background_pixmap)
        
    def paintEvent(self,event):
        painter = QPainter(self)
        painter.drawPixmap(self.rect(), self.background_pixmap)

    def on_register(self):
        
        new_username = self.textbox_new_username.text()
        new_password = self.textbox_new_password.text()
        new_phone = self.textbox_new_phone.text()
        new_address = self.textbox_new_address.text()
        new_city = self.textbox_new_city.text()
        new_state = self.textbox_new_state.text()
        new_zipcode = self.textbox_new_zipcode.text()
        new_country = self.textbox_new_country.text()
        
        if not all([new_username, new_password, new_phone, new_address, new_city, new_state, new_zipcode, new_country]):
            QMessageBox.warning(self, 'Register', 'Please fill in all the fields.')
            return
        
        
        conn = make_connection('invictus_db.ini')
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM cred WHERE username = %s", (new_username,))
        if cursor.fetchone():
            QMessageBox.warning(self, 'Register', 'Username already exists.')
        else:
            cursor.execute("INSERT INTO cred (username, _password) VALUES (%s, %s)", 
                           (new_username, new_password))
            
            cursor.execute( "SET @nextCustomerNumber = (SELECT MAX(customerNumber) + 1 FROM customers)")
            
            cursor.execute ( """  
                                INSERT INTO customers (
                                    customerNumber,
                                    customerName,
                                    phone,
                                    addressLine1,
                                    city,
                                    state,
                                    postalCode,
                                    country
                                    )
                                    VALUES ( @nextCustomerNumber,%s, %s,%s, %s,%s,%s, %s)""",
                                         
                                         (new_username,new_phone,  new_address, new_city, 
                                        new_state, 
                                        new_zipcode,
                                        new_country))
            
            
            conn.commit()
            QMessageBox.information(self, 'Register', 'Registration successful!')
        
        self.close()
            

    def show_dialog(self):
        self.exec_()

## Employee Login Page 

In [5]:
class EmpLoginPage(QDialog):
    def __init__(self):
        super().__init__()
        uic.loadUi('EmpLoginPage.ui', self)
        self.textbox_password.setEchoMode(QLineEdit.Password)
        self.button_login.clicked.connect(self.on_emplogin)
        self.employee_welcome_page = employee_welcome_page()
        self.background_pixmap = QPixmap('background_image.png')
        self.paintEvent(self.background_pixmap)
        
        self.backbutton_emp.clicked.connect(self.show_Login_page)

   
    
    def show_Login_page(self):
       
        self.close()
        self._show_Login_page = LoginPage()  
        self._show_Login_page.show_dialog()  
        
        
    def paintEvent(self,event):
        painter = QPainter(self)
        painter.drawPixmap(self.rect(), self.background_pixmap)

    def on_emplogin(self):
        username_employee = self.textbox_username.text()
        password = self.textbox_password.text()
        conn = make_connection('invictus_db.ini')
        cursor = conn.cursor()
        cursor.execute("SELECT emp_password FROM empcred WHERE emp_name = %s", (username_employee,))
        result = cursor.fetchone()

        if result and result[0] == password:
            
            self.employee_welcome_page.show_dialog()
        else:
            QMessageBox.warning(self, 'Login', 'Incorrect username or password.')
        
    def show_dialog(self):
        self.exec_()

## Welcome Page 

In [6]:

class WelcomePage(QDialog):
    def __init__(self,username=None):
        super().__init__()
        
        uic.loadUi('WelcomePage.ui', self)
        
        self._initialize_productlist_menu()
        self.productlist.currentIndexChanged.connect(self._initialize_querybutton)
        self._initialize_warehousetable()
        self._initialize_producttable()
        self._initialize_oldorderstable()
       
        self.neworderbutton.clicked.connect(self._show_neworder)
        self.oldorderbutton.clicked.connect(self._show_oldorder)
        self.oldorderbutton.clicked.connect(self._show_oldorderstable)

        
        self.querybutton.clicked.connect(self.enter_product_availability)
        self.querybutton.clicked.connect(self.enter_product_details)
        self.order_button.clicked.connect(self.place_order)
        
        
        
        self.producttable.hide()
        self.productlist.hide()
        self.warehousetable.hide()
        self.order_button.hide()
        self.querybutton.hide()
        self.previousorderslabel.hide()
        self.neworderslabel.hide()
        self.oldorderstable.hide()
        
        self.logoutbutton.clicked.connect(self._show_Login_page)
        
    def set_username(self, username):
        self.uname.setText(f'Welcome {username}')

    def _show_neworder(self):
        
        self.oldorderstable.hide()
        self.previousorderslabel.hide()
        
        self.producttable.setVisible(not self.producttable.isVisible())
        self.productlist.setVisible(not self.productlist.isVisible())
        self.warehousetable.setVisible(not self.warehousetable.isVisible())
        self.order_button.setVisible(not self.order_button.isVisible())
        self.querybutton.setVisible(not self.querybutton.isVisible())
        self.neworderslabel.setVisible(not self.neworderslabel.isVisible())
        
    
    def _show_oldorder(self):
        self.neworderslabel.hide()
        self.producttable.hide()
        self.productlist.hide()
        self.warehousetable.hide()
        self.order_button.hide()
        self.querybutton.hide()
        
        self.oldorderstable.setVisible(not self.oldorderstable.isVisible())
        self.previousorderslabel.setVisible(not self.previousorderslabel.isVisible())
   
        
    def _initialize_productlist_menu(self) :
        
        conn=make_connection('invictus_db.ini')
        cursor = conn.cursor()
        
        sql = """ select distinct productName  
            from products
            order by productName """
         
        cursor.execute(sql)
        rows=cursor.fetchall()
        
        for row in rows:
            self.productlist.addItem(row[0],row)
                 

        cursor.close()
        conn.close()
        
    def _initialize_querybutton(self):
        
        
        self.querybutton.setChecked(False)
        
    
    def _initialize_warehousetable(self):
        
        """
        Clear the table and set the column headers.
        """
        self.warehousetable.clear()

        col = ['warehouse' , 'availability']
        
        self.warehousetable.setHorizontalHeaderLabels(col)
        
    
    
    def _initialize_producttable(self):
        
        """
        Clear the table and set the column headers.
        """
        self.producttable.clear()

        
        col = ['product','Line', 'Vendor',  'price', 'Description']
        
        self.producttable.setHorizontalHeaderLabels(col)
        
    
    def _adjust_column_widths(self):
        
        header = self.producttable.horizontalHeader();
        header.setSectionResizeMode(0, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(1, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(2, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(4, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(3, QHeaderView.ResizeToContents)
    
    
    def enter_product_details(self):
        
        product_row = self.productlist.currentData()
        product=product_row[0]
        
        conn=make_connection('invictus_db.ini')
        cursor = conn.cursor()
        
        sql = f"""SELECT  productName as product, productLine as Line, productVendor as Vendor, 
                listPrice as price, productDescription as Description
                        FROM products
                        WHERE productName = '{product}'
                """
        
        cursor.execute(sql)
        rows=cursor.fetchall()
        
        row_index=0
        for row in rows:
            column_index=0
            for data in row:
                item = QTableWidgetItem(str(data))
                
                
                self.producttable.setItem(row_index, column_index, item)
                
                column_index += 1

            row_index += 1
    
        self._adjust_column_widths()
        

    def enter_product_availability(self):
        
        
        product_row = self.productlist.currentData()
        product=product_row[0]
        
        
        conn=make_connection('invictus_db.ini')
        cursor = conn.cursor()
        
        sql = f""" SELECT
                w.warehouseName,
                CASE
                    WHEN p.quantityInStock > 0 AND p.productName = '{product}' THEN 'yes'
                    ELSE 'no'
                END AS stockStatus
            FROM
                invictus_db.warehouses w
            LEFT JOIN invictus_db.products p 
            
            ON  w.warehouseCode = p.warehouseCode AND p.productName = '{product}'
            order by stockStatus desc
            """
        
        cursor.execute(sql)
        rows=cursor.fetchall()
        
        row_index=0
        for row in rows:
            column_index=0
            for data in row:
                item = QTableWidgetItem(str(data))
                self.warehousetable.setItem(row_index,column_index,item)
                column_index += 1

            row_index += 1
                                          
                                          
                                          
    def place_order(self):
        
        product_row = self.productlist.currentData()
        product=product_row[0]
        username = self.uname.text().replace('Welcome ', '' ).strip()
        
        conn=make_connection('invictus_db.ini')
        cursor = conn.cursor()
        
        cursor.execute(f"""INSERT INTO placed_order_records (placed_order_date,
                                 placed_productName ,
                                 placed_line ,
                                 placed_vendor,
                                 placed_price,
                                 username,
                                 product_code) 
                           
                           SELECT  CURRENT_DATE(),productName , productLine , productVendor ,listPrice, %s,productCode
                            FROM products
                            WHERE productName = %s
                        """,  (username,product))
        conn.commit()
        
        
        message_box = QMessageBox()
        message_box.setStyleSheet("QLabel{color: white;} QPushButton{ color: white; }")
        message_box.information(self, 'Place Order', 'Order Placed successfully!')
        
        
    def _initialize_oldorderstable(self):
        
        """
        Clear the table and set the column headers.
        """
        self.oldorderstable.clear()

        col = ['order_no','order_date', 'product',  'product_line', 'amount']
        
        self.oldorderstable.setHorizontalHeaderLabels(col)
        
        
        
        
    def _show_oldorderstable(self,username):
        
        
        
        conn=make_connection('invictus_db.ini')
        cursor = conn.cursor()
        
        cursor.execute( f""" SELECT  orderdetails.orderNumber as order_number, orders.orderDate as Date, productName as product, productLine as Line, 
                        quantityOrdered*sellingPrice as amount
                        FROM products, orderdetails, orders, customers c
                        WHERE products.productCode = orderdetails.productCode
                        and orders.orderNumber =orderdetails.orderNumber
                        and c.customerNumber = orders.customerNumber
                        and c.customerName = '{username}'
                        order by orders.orderDate DESC
                        """)

        
        rows=cursor.fetchall()
        
        row_index=0
        for row in rows:
            column_index=0
            for data in row:
                item = QTableWidgetItem(str(data))
                self.oldorderstable.setItem(row_index,column_index,item)
                column_index += 1

            row_index += 1
        
        
    def _show_Login_page(self):
        self.close()
        self._login_page = LoginPage()
        self._login_page.show_dialog()
                            
        
    def show_dialog(self):
        self.exec_()
        

        


## Employee Welcome Page

In [7]:
class employee_welcome_page(QDialog):
    
    def __init__(self):
        
        super().__init__()
        
        uic.loadUi('employee_welcome_page.ui', self)
        

        self.dashboards_button.clicked.connect(self._show_dashboards)
        self.order_requests_button.clicked.connect(self._show_orders)

        self.load_image()
        
        
        
        self.logoutbutton_2.clicked.connect(self._show_Emp_Login_page) 
    
    def _show_Emp_Login_page(self):
        self.close()
        self._emp_login_page = EmpLoginPage()
        self._emp_login_page.show_dialog()
        
    def load_image(self):
        self.imageLabel = self.findChild(QLabel, 'imagelabel')  
        pixmap = QPixmap('icon.png')  
        self.imageLabel.setPixmap(pixmap.scaled(self.imageLabel.width(), self.imageLabel.height(), 
                                                Qt.KeepAspectRatio))

    
    def _show_dashboards(self):
        self.dashboards = AnalysisPage()
        self.dashboards.show_dialog()
        
    def _show_orders(self):
        
        self.orders = order_list_page()

    def show_dialog(self):
        self.exec_()

    

## Order List Page 

In [8]:
class order_list_page(object):
    
    
    def __init__(self):
        super().__init__()
   
        self.Dialog = QDialog()
        self.setupUi(self.Dialog)  
        self.Dialog.setWindowFlags(self.Dialog.windowFlags() | Qt.WindowStaysOnTopHint)
        self.Dialog.setModal(True)
        self.Dialog.showMaximized()

    
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(417, 300)
        

        self.tableWidget = QtWidgets.QTableWidget(Dialog)
        self.tableWidget.setGeometry(QtCore.QRect(460, 290, 1001, 631))
        self.tableWidget.setColumnCount(7)  
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setRowCount(0)
        
        
    
        self.moveButton = QtWidgets.QPushButton(Dialog)
        self.moveButton.setGeometry(QtCore.QRect(1200, 130, 131, 41))
        self.moveButton.setStyleSheet("background-color: rgb(186, 215, 255);")
        self.moveButton.setText("Process Order")
        self.moveButton.clicked.connect(self.move_selected_records_to_orderdetails)


        self.label = QtWidgets.QLabel(Dialog)
        self.label.setGeometry(QtCore.QRect(690, 140, 231, 21))
        self.label.setObjectName("label")
        self.label.setStyleSheet("font-size: 20pt; font-family: Monaco;")


        
        self.pushButton = QtWidgets.QPushButton(Dialog)
        self.pushButton.setGeometry(QtCore.QRect(1050, 130, 131, 41))
        self.pushButton.setStyleSheet("background-color: rgb(186, 215, 255);")
    

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)

        
        self.populate_tableWidget()
        
        self.pushButton.clicked.connect(self._show_employee_welcome_page)

    def _show_employee_welcome_page(self):
        
        self.Dialog.close()
        self._employee_welcome_page = employee_welcome_page()
        
        self._employee_welcome_page.show_dialog()
        
        
    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Dialog"))
        self.label.setText(_translate("Dialog", "Requested Orders"))
        self.label.setStyleSheet("font-size: 20pt; font-family: Monaco;")
        self.pushButton.setText(_translate("Dialog", "Back"))  # Set the text for the Back button
        
        
        
    def _adjust_column_widths(self):
        
        header = self.tableWidget.horizontalHeader();
        header.setSectionResizeMode(0, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(1, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(2, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(3, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(4, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(5, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(6, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(7, QHeaderView.Stretch)

    def populate_tableWidget(self):
        
        connection = pymysql.connect(host='IES-ADS-ClassDB.sjsu.edu', user='invictus_user', password='Guava_449', db='invictus_db')

        with connection.cursor() as cursor:
            
            query = "SELECT * FROM invictus_db.placed_order_records;"
            cursor.execute(query)
            result = cursor.fetchall()

        
        column_names = [i[0] for i in cursor.description]

        
        self.tableWidget.setColumnCount(len(column_names))
        self.tableWidget.setHorizontalHeaderLabels(column_names)
        self.tableWidget.setRowCount(len(result))
        for row_number, row_data in enumerate(result):
            for column_number, data in enumerate(row_data):
                item = QtWidgets.QTableWidgetItem(str(data))
                item.setForeground(QtGui.QBrush(QtGui.QColor(0,  0, 0)))  
                self.tableWidget.setItem(row_number, column_number, item)
               
        checkbox_header = QtWidgets.QTableWidgetItem("Select")
        self.tableWidget.setColumnCount(self.tableWidget.columnCount() + 1)  # Increment the column count
        self.tableWidget.setHorizontalHeaderItem(self.tableWidget.columnCount() - 1, checkbox_header)

        for row_number, row_data in enumerate(result):
            for column_number, data in enumerate(row_data):
                item = QtWidgets.QTableWidgetItem(str(data))
                item.setForeground(QtGui.QBrush(QtGui.QColor(0,  0, 0)))  
                self.tableWidget.setItem(row_number, column_number, item)

            
            checkbox_item = QtWidgets.QTableWidgetItem()
            checkbox_item.setFlags(QtCore.Qt.ItemIsUserCheckable | QtCore.Qt.ItemIsEnabled)
            checkbox_item.setCheckState(QtCore.Qt.Unchecked)
            self.tableWidget.setItem(row_number, self.tableWidget.columnCount() - 1, checkbox_item)
            

    def move_selected_records_to_orderdetails(self):
        connection = pymysql.connect(host='IES-ADS-ClassDB.sjsu.edu', user='invictus_user', password='Guava_449', db='invictus_db')
        with connection.cursor() as cursor:
            selected_items_exist = False
            for row_number in range(self.tableWidget.rowCount()):
                checkbox_item = self.tableWidget.item(row_number, self.tableWidget.columnCount() - 1)
                if checkbox_item and checkbox_item.checkState() == QtCore.Qt.Checked:
                    selected_items_exist = True
                    
                    selected_row_data = [self.tableWidget.item(row_number, col).text() for col in range(self.tableWidget.columnCount() - 1)]
                    

                    
                    query = """INSERT INTO orderdetails (orderNumber, productCode, quantityOrdered, sellingPrice, orderLineNumber)
                               VALUES (%s, %s, %s, %s, %s);"""
                    
                    cursor.execute(query, (selected_row_data[0], selected_row_data[7], 1, selected_row_data[5], 1))

                   
                    
                    cursor.execute(f"SELECT customerNumber FROM customers WHERE customerName = '{selected_row_data[6]}'")
                    x = cursor.fetchone()
                     
                    
                    cursor.execute('SELECT CURRENT_DATE()')
                    y = cursor.fetchone()
                    
                    

                    sql = """INSERT INTO orders (orderNumber, orderDate, requiredDate, shippedDate, status, comments, customerNumber)
                              VALUES (%s, %s, %s, %s, %s,%s,%s);"""
                    
                    cursor.execute(sql,(selected_row_data[0],selected_row_data[1],y[0] , y[0],  'NONE', 'NONE',x[0]))
                    
                    self.tableWidget.removeRow(row_number)
                    
                    cursor.execute(f'DELETE FROM placed_order_records WHERE placed_order_id = {selected_row_data[0]}')       

        connection.commit()

        self._adjust_column_widths()
        
        connection.close()
        if selected_items_exist:
            QMessageBox.information(self.Dialog, 'Process Order', 'Order Processed!')
        else:
            QMessageBox.warning(self.Dialog, 'Process Order', 'Please select a product to process!')

        
         
        
        
    def show_dialog(self):
        self.Dialog.exec_()


## Analytics Page 

In [9]:
class AnalysisPage(QDialog):
    def __init__(self):
        super().__init__()
        
        uic.loadUi('analytics_welcome.ui', self)

        
        self.sales_button.clicked.connect(self._show_sales_dashboard)
        self.products_button.clicked.connect(self._show_stocks_dashboard)
        self.warehouse_button.clicked.connect(self._show_warehouse_dashboard)
        
        
        self.background_pixmap = QPixmap('analytics.png')
        self.paintEvent(self.background_pixmap)
        self.analyticsbackButton.clicked.connect(self._show_employee_welcome_page)
        
        
    def _show_employee_welcome_page(self):
        
        self.close()
        self._employee_welcome_page = employee_welcome_page()
        
        self._employee_welcome_page.show_dialog()
        
    
    def paintEvent(self,event):
        painter = QPainter(self)
        painter.setOpacity(0.5)
        painter.drawPixmap(self.rect(), self.background_pixmap)
        

    

    def _show_warehouse_dashboard(self):
        self.warehouse_dashboard = warehouse_dashboard()
        

    def show_dialog(self):
        self.exec_()

    def _show_sales_dashboard(self):
        self.sales_dashboard = sales_dashboard()
        

    def _show_stocks_dashboard(self):
        self.stocks_dashboard = stocks_dashboard()
        

## Warehouse Dashboard 

In [10]:
class warehouse_dashboard(object):
    
    def __init__(self):
        super().__init__()
       
        self.Dialog = QDialog()
        
        self.setupUi(self.Dialog)
        self.add_bar_plot(self.Dialog)
        self.Dialog.setWindowFlags(self.Dialog.windowFlags() | Qt.WindowStaysOnTopHint)
        self.Dialog.setModal(True)  
        self.Dialog.showMaximized()
    
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(990, 739)
        
        self.verticalLayout = QVBoxLayout(Dialog)
        self.verticalLayout.setObjectName("verticalLayout")
        self.frame = QFrame(Dialog)
        self.frame.setFrameShape(QFrame.StyledPanel)
        self.frame.setFrameShadow(QFrame.Raised)
        self.frame.setObjectName("frame")
        self.verticalLayout_2 = QVBoxLayout(self.frame)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.title_container = QFrame(self.frame)
        self.title_container.setFrameShape(QFrame.StyledPanel)
        self.title_container.setFrameShadow(QFrame.Raised)
        self.title_container.setObjectName("title_container")
        self.horizontalLayout = QHBoxLayout(self.title_container)
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.plot_title = QLabel(self.title_container)
        self.plot_title.setGeometry(QtCore.QRect(10, 10, 932, 89))
        font = QtGui.QFont()
        font.setFamily("Verdana")
        font.setPointSize(9)
        font.setBold(False)
        font.setItalic(False)
        font.setWeight(9)
        self.plot_title.setFont(font)
        self.plot_title.setStyleSheet("font: 75 15pt \"Verdana\";\n"
                                     "bold")
        self.plot_title.setAlignment(QtCore.Qt.AlignCenter)
        self.plot_title.setObjectName("plot_title")
        self.verticalLayout_2.addWidget(self.title_container)
        self.wh_box_container = QFrame(self.frame)
        self.wh_box_container.setFrameShape(QFrame.StyledPanel)
        self.wh_box_container.setFrameShadow(QFrame.Raised)
        self.wh_box_container.setObjectName("wh_box_container")
        self.wh_box = QComboBox(self.wh_box_container)
        self.wh_box.setGeometry(QtCore.QRect(20, 0, 381, 61))
        
        self.wh_box.setObjectName("wh_box")
        self.verticalLayout_2.addWidget(self.wh_box_container)
        self.wh_box.currentIndexChanged.connect(self.on_wh_box_changed)
        self.loadComboBoxData()
        self.wh_infobox = QLabel(self.frame)  
        self.wh_infobox.setFrameShape(QFrame.StyledPanel)
        self.wh_infobox.setFrameShadow(QFrame.Raised)
        self.wh_infobox.setObjectName("wh_infobox")
        
        self.verticalLayout_2.addWidget(self.wh_infobox)
        self.verticalLayout.addWidget(self.frame)
        self.plot_box = QFrame(Dialog)
        self.plot_box.setFrameShape(QFrame.StyledPanel)
        self.plot_box.setFrameShadow(QFrame.Raised)
        self.plot_box.setObjectName("plot_box")
        self.verticalLayout.addWidget(self.plot_box)

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)
        
        

        
        
    
        
        self.title_container = QFrame(self.frame)
        self.title_container.setFrameShape(QFrame.StyledPanel)
        self.title_container.setFrameShadow(QFrame.Raised)
        self.title_container.setObjectName("title_container")
        self.horizontalLayout = QHBoxLayout(self.title_container)
        self.horizontalLayout.setObjectName("horizontalLayout")

        
        spacer = QSpacerItem(40, 20, QSizePolicy.Expanding, QSizePolicy.Minimum)
        self.horizontalLayout.addItem(spacer)

        

        
        self.backPageButton = QPushButton("Back", self.title_container)
        self.backPageButton.setObjectName("backPageButton")
        
        self.backPageButton.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        self.backPageButton.setStyleSheet("background-color: rgb(186, 215, 255);")

       
        self.horizontalLayout.addWidget(self.backPageButton)

        
        self.backPageButton.clicked.connect(self.open_dashboard)

        self.verticalLayout_2.addWidget(self.title_container)
    
    
        
    def open_dashboard(self):
        
        self.Dialog.close()
        self.dashboardPage = AnalysisPage()  
        self.dashboardPage.show_dialog()  
        
        
        

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Dialog"))
        self.plot_title.setText(_translate("Dialog", "Warehouse Performance"))

    def add_bar_plot(self, Dialog):
        connection = make_connection(config_file='invictus_db.ini')
        cursor = connection.cursor()
        try:
            with connection.cursor() as cursor:
                # Execute the SQL query
                sql_query = """
                SELECT w.warehouseName as Warehouse, COUNT(o.orderNumber) as Orders
                FROM orderdetails o, warehouses w, products p
                WHERE w.warehouseCode = p.warehouseCode AND o.productCode = p.productCode
                GROUP BY w.warehouseCode
                """
                cursor.execute(sql_query)
                result = cursor.fetchall()

                
                df = pd.DataFrame(result, columns=["Warehouse", "Orders"])

                
                figure = Figure()
                canvas = FigureCanvas(figure)
                axis = figure.add_subplot(111)

                
                axis.bar(df["Warehouse"], df["Orders"], color='skyblue')
                axis.set_title('Warehouse vs Orders')
                axis.set_xlabel('Warehouse')
                axis.set_ylabel('Orders')

                
                self.plot_box.setLayout(QVBoxLayout())
                self.plot_box.layout().addWidget(canvas)
                self.plot_box.show()
        finally:
            
            connection.close()

    def loadComboBoxData(self):
        connection = make_connection(config_file='invictus_db.ini')
        cursor = connection.cursor()
        try:
            with connection.cursor() as cursor:
                
                sql_query = "SELECT warehouseName FROM warehouses"
                cursor.execute(sql_query)
                results = cursor.fetchall()

                
                self.wh_box.clear()

                
                for row in results:
                    self.wh_box.addItem(row[0])
        except Exception as e:
            print("Error loading ComboBox data:", e)

    def on_wh_box_changed(self, index):
        
        selected_warehouse = self.wh_box.itemText(index)
        self.updateInfoBox(selected_warehouse)

    def updateInfoBox(self, warehouse_name):
        connection = make_connection(config_file='invictus_db.ini')
        cursor = connection.cursor()
        try:
            with connection.cursor() as cursor:
                
                
                
                sql_query_capacity = """SELECT CONCAT('In warehouse ', r.warehouseName, 
                                              ', the following products have generated sales:\n ',
                                              GROUP_CONCAT(r.productDetails SEPARATOR '\n' ),
                                              '.') AS warehouseSalesSummary
                                FROM (
                                    SELECT r.warehouseName, 
                                           CONCAT(r.productName, ' with a stock of ', r.quantityInStock, 
                                                  ' units, has generated sales worth $', 
                                                  FORMAT(SUM(o.quantityOrdered * o.sellingPrice), 2)) AS productDetails
                                    FROM RankedProductsView AS r
                                    INNER JOIN products AS p ON r.productName = p.productName
                                    INNER JOIN orderdetails AS o ON p.productCode = o.productCode
                                    WHERE r.rank_q IN (1, 2)
                                    GROUP BY r.warehouseName, r.productName, r.quantityInStock
                                ) AS r
                                WHERE  r.warehouseName = %s
                                GROUP BY r.warehouseName
                                ORDER BY r.warehouseName; """

                
                
                


                cursor.execute(sql_query_capacity, (warehouse_name,))
                capacity_result = cursor.fetchone()
                if capacity_result:
                    
                    self.wh_infobox.setText(f" {capacity_result[0]}")
                else:
                    self.wh_infobox.setText("No capacity data available for this warehouse.\n")

        except Exception as e:
            print("Error fetching data for info box:", e)
            self.wh_infobox.setText("Error loading data.\n")

        finally:
            if 'connection' in locals() and connection:
                connection.close()  # Ensure connection is closed
    
    def show_dialog(self):
        self.Dialog.exec_()
        



## Sales Dashboard 

In [11]:
class sales_dashboard(object):
    
    
    def __init__(self):
        super().__init__()

        
        self.Dialog = QDialog()
        self.setupUi(self.Dialog)  
        
        self.Dialog.setWindowFlags(self.Dialog.windowFlags() | Qt.WindowStaysOnTopHint)
        self.Dialog.setModal(True)  
        self.Dialog.showMaximized()
        
        
    def setupUi(self, Dialog):
        
        Dialog.setObjectName("Dialog")
        Dialog.resize(1070, 338)
        

        self.rolluptable = QtWidgets.QTableWidget(Dialog)
        self.rolluptable.setGeometry(QtCore.QRect(80, 60, 560, 600))
        self.rolluptable.setRowCount(250)
        self.rolluptable.setColumnCount(4)
        self.rolluptable.setObjectName("rolluptable")
       

        self.label = QtWidgets.QLabel(Dialog)
        self.label.setGeometry(QtCore.QRect(600, 20, 201, 16))
        
        self.label.setObjectName("label")

        self.Yearlysales = QtWidgets.QFrame(Dialog)
        self.Yearlysales.setGeometry(QtCore.QRect(700, 40, 620, 340))
        self.Yearlysales.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.Yearlysales.setFrameShadow(QtWidgets.QFrame.Raised)
        self.Yearlysales.setObjectName("Yearlysales")

        self.productlinessales = QtWidgets.QFrame(Dialog)
        self.productlinessales.setGeometry(QtCore.QRect(700, 380, 620, 320))
        self.productlinessales.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.productlinessales.setFrameShadow(QtWidgets.QFrame.Raised)
        self.productlinessales.setObjectName("productlinessales")

        
        self.populate_rolluptable()

        
        self.add_line_chart(self.Yearlysales)

       
        self.add_bar_chart(self.productlinessales)

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)
        
        
        self.backButton = QPushButton(Dialog)
        self.backButton.setText("Back")
        self.backButton.setGeometry(QtCore.QRect(800, 720, 180, 40)) 
        
        self.backButton.setStyleSheet("background-color: rgb(186, 215, 255);")

    
        self.backButton.clicked.connect(self.open_dashboard)
    
    def open_dashboard(self):
        
        self.Dialog.close()
        self.dashboardPage = AnalysisPage()  
        self.dashboardPage.show_dialog()  
        
        

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Sales Dashboard"))
        self.label.setText(_translate("Dialog", "Sales Dashboard"))

    def populate_rolluptable(self):
        
        connection = pymysql.connect(host='IES-ADS-ClassDB.sjsu.edu', user='invictus_user', password='Guava_449', db='invictus_wh')

        with connection.cursor() as cursor:
           
            query = """SELECT DISTINCT c.customerName, CONCAT(e.firstName, ' ', e.lastName) AS employeeFullName, 
                       pf.officeCode, SUM(amount) as salesUSD
                       FROM payments_facttable pf
                       JOIN employees e ON pf.employeeNumber = e.employeeNumber
                       JOIN customer c ON pf.customerNumber = c.customerNumber
                       JOIN office o ON pf.officeCode = o.officeCode
                       GROUP BY c.customerName, employeeFullName, pf.officeCode WITH ROLLUP;"""
            cursor.execute(query)
            result = cursor.fetchall()

        
        self.rolluptable.setRowCount(len(result))
        self.rolluptable.setHorizontalHeaderLabels(['Customer Name', 'Employee Name', 'Office Code', 'Sales (USD)'])
        for row_number, row_data in enumerate(result):
            for column_number, data in enumerate(row_data):
                item = QtWidgets.QTableWidgetItem(str(data))
                item.setForeground(QtGui.QBrush(QtGui.QColor(0, 0, 0)))  # Set text color to white
                self.rolluptable.setItem(row_number, column_number, item)

       
        connection.close()

    def add_line_chart(self, frame):
        
        connection = pymysql.connect(host='IES-ADS-ClassDB.sjsu.edu', user='invictus_user', password='Guava_449', db='invictus_wh')

        with connection.cursor() as cursor:
            
            query = "SELECT SUM(dollarsSold), YEAR(orderDate) as Year_ FROM sales_facttable GROUP BY Year_"
            cursor.execute(query)
            result = cursor.fetchall()

        
        sales = [row[0] for row in result]
        years = [row[1] for row in result]

       
        connection.close()

        
        figure = Figure()
        canvas = FigureCanvas(figure)
        axis = figure.add_subplot(111)

       
        axis.plot(years, sales)
        axis.set_title('Yearly Sales', fontsize=8)
        axis.set_xlabel('Year', fontsize=6)
        axis.set_ylabel('Sales (USD)', fontsize=6)
        
        for label in axis.get_xticklabels():
            label.set_fontsize(6)
        for label in axis.get_yticklabels():
            label.set_fontsize(6)
        figure.tight_layout()

        
        layout = QtWidgets.QVBoxLayout(frame)
        layout.addWidget(canvas)

    def add_bar_chart(self, frame):
       
        connection = pymysql.connect(host='IES-ADS-ClassDB.sjsu.edu', user='invictus_user', password='Guava_449', db='invictus_wh')

        with connection.cursor() as cursor:
           
            query = """SELECT p.productLine, SUM(s.quantityOrdered) AS totalQuantityOrdered
                       FROM product p
                       JOIN sales_facttable s ON p.productCode = s.productCode
                       GROUP BY p.productLine
                       ORDER BY totalQuantityOrdered DESC;"""
            cursor.execute(query)
            result = cursor.fetchall()

        
        product_lines = [row[0] for row in result]
        quantities = [row[1] for row in result]

        
        connection.close()

        
        figure = Figure()
        canvas = FigureCanvas(figure)
        axis = figure.add_subplot(111)

        
        axis.bar(product_lines, quantities)
        axis.set_title('Product Line Sales', fontsize=8)
        axis.set_xlabel('Product Line', fontsize=6)
        axis.set_ylabel('Total Quantity Ordered', fontsize=6)
        for label in axis.get_xticklabels():
            label.set_fontsize(6)
        for label in axis.get_yticklabels():
            label.set_fontsize(6)

        
        layout = QtWidgets.QVBoxLayout(frame)
        layout.addWidget(canvas)


    def show_dialog(self):
        self.Dialog.exec_()
   


##  Stocks Dashboard

In [12]:

class stocks_dashboard(object):
    
    def __init__(self):
        super().__init__()

        
        self.Dialog = QDialog()
        self.setupUi(self.Dialog) 
        self.Dialog.setWindowFlags(self.Dialog.windowFlags() | Qt.WindowStaysOnTopHint)
        self.Dialog.setModal(True)  
        self.Dialog.showMaximized()

    
    
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(1025, 561)
        self.label = QtWidgets.QLabel(Dialog)
        self.label.setGeometry(QtCore.QRect(600, 10, 500, 31))
        self.label.setObjectName("label")
        self.Northframe = QtWidgets.QFrame(Dialog)
        self.Northframe.setGeometry(QtCore.QRect(80, 100, 560, 211))
        self.Northframe.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.Northframe.setFrameShadow(QtWidgets.QFrame.Raised)
        self.Northframe.setObjectName("Northframe")
        self.Eastframe = QtWidgets.QFrame(Dialog)
        self.Eastframe.setGeometry(QtCore.QRect(80, 340, 560, 231))
        self.Eastframe.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.Eastframe.setFrameShadow(QtWidgets.QFrame.Raised)
        self.Eastframe.setObjectName("Eastframe")
        self.Southframe = QtWidgets.QFrame(Dialog)
        self.Southframe.setGeometry(QtCore.QRect(700, 100, 620, 211))
        self.Southframe.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.Southframe.setFrameShadow(QtWidgets.QFrame.Raised)
        self.Southframe.setObjectName("Southframe")
        self.Westframe = QtWidgets.QFrame(Dialog)
        self.Westframe.setGeometry(QtCore.QRect(700, 340, 620, 231))
        self.Westframe.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.Westframe.setFrameShadow(QtWidgets.QFrame.Raised)
        self.Westframe.setObjectName("Westframe")


        
        self.add_pie_chart(Dialog, self.Northframe, "a")  # North
        self.add_pie_chart(Dialog, self.Southframe, "d")  # South
        self.add_pie_chart(Dialog, self.Eastframe, "b")   # East
        self.add_pie_chart(Dialog, self.Westframe, "c")   # West

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)
        
        
        
        self.backButton = QPushButton(Dialog)
        self.backButton.setText("Back")
        self.backButton.setGeometry(QtCore.QRect(800, 720, 180, 40))  
        self.backButton.setStyleSheet("background-color: rgb(186, 215, 255);")
        
        
   
        self.backButton.clicked.connect(self.open_dashboard)
    
    def open_dashboard(self):
        
        self.Dialog.close()
        self.dashboardPage = AnalysisPage()  
        self.dashboardPage.show_dialog()  
        
        

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Dialog"))
        self.label.setText(_translate("Dialog", "Product Vs Quantity"))


    def add_pie_chart(self, Dialog, frame, warehouse_code):
        warehouse_names = {"a": "North", "b": "East", "c": "West", "d": "South"}
        
        connection = pymysql.connect(host='IES-ADS-ClassDB.sjsu.edu', user='invictus_user', password='Guava_449', db='invictus_db')

        with connection.cursor() as cursor:
            
            query = f"SELECT productName, quantityInStock FROM products WHERE warehouseCode='{warehouse_code}' GROUP BY productCode,warehouseCode ORDER BY quantityInStock desc LIMIT 5;"
            cursor.execute(query)
            result = cursor.fetchall()

            
            quantities = [row[1] for row in result]
            products = [row[0] for row in result]

        
        connection.close()

        
        figure = Figure()
        canvas = FigureCanvas(figure)
        axis = figure.add_subplot(111)
        
        
        patches, texts, autotexts = axis.pie(quantities, labels=products, autopct='%1.1f%%', startangle=90)
        for text in texts:
            text.set_fontsize(7)
        warehouse_full_name = warehouse_names.get(warehouse_code, "Unknown") + " Warehouse"
        axis.set_title(f'Products in {warehouse_full_name}')

        
        layout = QtWidgets.QVBoxLayout(frame)
        layout.addWidget(canvas)
        
    def show_dialog(self):
        self.Dialog.exec_()



## final run

In [13]:
def main():
    app = QApplication(sys.argv)
    login_page = LoginPage()
   
    
    login_page.show()
    sys.exit(app.exec_())
    super().closeEvent()

if __name__ == '__main__':
    main()


Error fetching data for info box: 'warehouse_dashboard' object has no attribute 'wh_infobox'


AttributeError: 'warehouse_dashboard' object has no attribute 'wh_infobox'

SystemExit: 0

C:\Users\kothu\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
